# Creating a feature matrix from a DB Query

In [1]:
import sys
sys.path.append('/Users/celia/GitHub/mouse_bandit/data_preprocessing_code')
sys.path.append('/Users/celia/GitHub/mouse_bandit')
import support_functions as sf
import numpy as np
import pandas as pd
import scipy as sp
import bandit_preprocessing as bp
from sklearn import preprocessing
import sys
import os
import ipywidgets as widgets
from IPython.display import display
%matplotlib inline 

## Retrieve names of sessions you want from the DB

In [18]:
#load in data base
db = pd.read_csv('/Users/celia/GitHub/mouse_bandit/celia_session_record.csv',index_col=0)

In [19]:
db.shape

(519, 18)

In [20]:
db.head(5)

,Owner,Session ID,Mouse ID,Date,Phase,Markov,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration
0,celia,01112018_Baby,Baby,2018-01-11,2.0,1,0.8,0.2,NaN,NaN,597.0,9.0,431.0,0.88,2.0,1.0,40.0,40.0
1,celia,01122018_Baby,Baby,2018-01-12,2.0,1,0.2,0.8,NaN,NaN,750.0,15.0,542.0,0.86,2.0,1.0,40.0,40.0
2,celia,01152018_Baby,Baby,2018-01-15,2.0,1,0.8,0.2,NaN,NaN,800.0,13.0,591.0,0.88,2.0,1.0,40.0,40.0
3,celia,08032017_Baby,Baby,2017-08-03,0.0,0,1.0,1.0,1000.0,1000.0,3.0,0.0,0.0,0.00,10.0,1.0,50.0,50.0
4,celia,08042017_Baby,Baby,2017-08-04,1.0,0,0.9,0.9,1000.0,1000.0,19.0,0.0,15.0,0.00,15.0,1.0,60.0,60.0


### Query all (80-20) sessions where performance > 0.7 and number of blocks exceeded 0 (no error)

In [6]:
r=db.copy()

In [21]:
#r = db.copy()
r = db[(db['Markov']==1)]
#r = db[((db['Left Reward Prob'] == 0.7) |  (db['Left Reward Prob'] == 0.3))].copy()
r = r[r['p(high Port)'] > 0.7].copy()
r = r[r['No. Blocks']>0.0].copy()
#r = r[r['Mouse ID'] == 'Mustard']
session_names = r['Session ID'].values

In [22]:
r.shape

(324, 18)

In [9]:
r2 = db[(db['Markov']==1)]
r2 = r[r['No. Blocks']>0.0].copy()
r2.shape

(303, 18)

In [10]:
np.min(r2['p(high Port)'])

0.70999999999999996

In [345]:
#r.to_csv(os.path.join(save_dir,'trials_markov_9010.csv'),index=True)

## load in csv files (from running exportTrials.m)

In [23]:
'''
load in trial data
'''
columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked',
           'Right Reward Prob','Left Reward Prob','Reward Given',
          'center_frame','decision_frame', 'Block ID']

root_dir = '/Users/celia/GitHub/mouse_bandit/data/trial_data'
save_dir = '/Users/celia/GitHub/mouse_bandit/data/processed_data'

trial_df = []

for session in session_names:
    full_name = session + '_trials.csv'
    
    path_name = os.path.join(root_dir,full_name)
    
    trial_df.append(pd.read_csv(path_name,names=columns))

mouse_ids = r['Mouse ID'].values

In [24]:
len(trial_df)

324

## convert into 1 feature matrix

In [25]:
progress = widgets.IntProgress(min=0, max=len(trial_df), step=1)
display(progress)
for i,df in enumerate(trial_df):
    curr_feature_matrix = bp.create_feature_matrix(df,10,mouse_ids[i],session_names[i],feature_names='Default')
    
    if i == 0:
        master_matrix = curr_feature_matrix.copy()
    else:
        master_matrix = master_matrix.append(curr_feature_matrix)
    progress.value+=1

In [58]:
master_matrix.tail(5)

,Mouse ID,Session ID,Trial,Block Trial,Block Reward,Port Streak,Reward Streak,10_Port,10_Reward,10_ITI,...,1_Port,1_Reward,1_ITI,1_trialDuration,0_ITI,0_trialDuration,Decision,Switch,Higher p port,Reward
796,Wine,10162017_Wine,807.0,11.0,5.0,1.0,1.0,1.0,1.0,1.387,...,1.0,1.0,1.295,0.315,1.310,0.294,1.0,0.0,0.0,1.0
797,Wine,10162017_Wine,808.0,12.0,6.0,2.0,2.0,1.0,0.0,1.377,...,1.0,1.0,1.310,0.294,1.297,0.316,1.0,0.0,0.0,0.0
798,Wine,10162017_Wine,809.0,13.0,6.0,3.0,-1.0,1.0,0.0,1.372,...,1.0,0.0,1.297,0.316,1.323,0.506,1.0,0.0,0.0,0.0
799,Wine,10162017_Wine,810.0,14.0,6.0,4.0,-2.0,1.0,0.0,1.208,...,1.0,0.0,1.323,0.506,1.838,0.570,0.0,1.0,1.0,0.0
800,Wine,10162017_Wine,811.0,15.0,6.0,1.0,-3.0,1.0,0.0,12.070,...,0.0,0.0,1.838,0.570,1.327,0.462,1.0,1.0,0.0,0.0


In [26]:
master_matrix.shape

(230819, 53)

In [103]:
master_matrix['Higher p port'].mean()

0.8428510090167454

## Save combined feature matrix  

In [27]:
master_matrix.to_csv(os.path.join(save_dir,'markov_master.csv'),index=True)